In [3]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [4]:
!python -V

Python 3.10.13


In [5]:
import pickle
import pandas as pd

In [20]:
import os

pwd = os.getcwd()

In [22]:
PATH = os.path.join(pwd,'mlops-zoomcamp/mlops-zoomcamp-2024/04-deployment/homework/')
PATH

'/Users/andrescardenashernandez/Library/CloudStorage/OneDrive-Anheuser-BuschInBev/Projects/mlops-zoomcamp/mlops-zoomcamp-2024/04-deployment/homework/'

In [23]:
with open(f'{PATH}/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [8]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    return df

In [9]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [10]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [11]:
y_pred

array([16.24590642, 26.1347962 , 11.88426424, ..., 11.59533603,
       13.11317847, 12.89999218])

In [12]:
import numpy as np

std_dev = np.std(y_pred)
std_dev

np.float64(6.247488852238703)

In [15]:
year = 2023
month=3

In [16]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [18]:
df_result = pd.DataFrame({'ride_id': df['ride_id'], 'predictions': y_pred})

In [24]:
output_file = f'{PATH}/output.parquet'


In [25]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [27]:
df_result.head()

,ride_id,predictions
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486


In [28]:
output_path = f'{PATH}/output/yellow/2023--03.parquet'
df_output = pd.read_parquet(output_path)

In [29]:
mean_predictions = df_output['predictions'].mean()
mean_predictions

np.float64(14.203865642696083)